In [110]:
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

In [111]:
df = pd.read_csv(r"C:\Users\Manish\Desktop\datasets\train.csv")

In [112]:
df.head()
no_rows = df.shape[0]
no_columns = df.shape[1]

In [113]:
df = df.drop(['customer_id','Name','security_no','referral_id'],axis = 1)

In [114]:
df.head()

,age,gender,region_category,membership_category,joining_date,joined_through_referral,preferred_offer_types,medium_of_operation,internet_option,last_visit_time,...,avg_time_spent,avg_transaction_value,avg_frequency_login_days,points_in_wallet,used_special_discount,offer_application_preference,past_complaint,complaint_status,feedback,churn_risk_score
0,18,F,Village,Platinum Membership,2017-08-17,No,Gift Vouchers/Coupons,?,Wi-Fi,16:08:02,...,300.63,53005.25,17.0,781.75,Yes,Yes,No,Not Applicable,Products always in Stock,2
1,32,F,City,Premium Membership,2017-08-28,?,Gift Vouchers/Coupons,Desktop,Mobile_Data,12:38:13,...,306.34,12838.38,10.0,NaN,Yes,No,Yes,Solved,Quality Customer Care,1
2,44,F,Town,No Membership,2016-11-11,Yes,Gift Vouchers/Coupons,Desktop,Wi-Fi,22:53:21,...,516.16,21027.00,22.0,500.69,No,Yes,Yes,Solved in Follow-up,Poor Website,5
3,37,M,City,No Membership,2016-10-29,Yes,Gift Vouchers/Coupons,Desktop,Mobile_Data,15:57:50,...,53.27,25239.56,6.0,567.66,No,Yes,Yes,Unsolved,Poor Website,5
4,31,F,City,No Membership,2017-09-12,No,Credit/Debit Card Offers,Smartphone,Mobile_Data,15:46:44,...,113.13,24483.66,16.0,663.06,No,Yes,Yes,Solved,Poor Website,5


In [115]:
df['last_visit_time']

0        16:08:02
1        12:38:13
2        22:53:21
3        15:57:50
4        15:46:44
           ...   
36987    04:14:05
36988    23:18:31
36989    03:50:25
36990    09:50:03
36991    01:39:52
Name: last_visit_time, Length: 36992, dtype: object

In [116]:
 #This step converts all the missing and irrelevant data  present in the dataframe to NaN
def convert_to_NaN(df):
    for column in df:
        df[column] = df[column].apply(lambda x: np.nan if x == '?' or x == 'Error' or x == 'No Information Available' or x == 'No reason specified' else x)        
    df.fillna(0,inplace = True)
    return df    

In [117]:
df = convert_to_NaN(df)

In [118]:
def convert_time(df):
    df['last_visit_time'] = pd.to_datetime(df['last_visit_time'],format = '%H:%M:%S')
    reference_time = ['23:59:59']*no_rows
    df['reference_time'] = reference_time
    df['reference_time'] = pd.to_datetime(df['reference_time'],format = '%H:%M:%S')
    df['seconds_since_last_visit'] = ( df['reference_time'] - df['last_visit_time'] ).dt.total_seconds()
    #df['seconds_since_last_visit']
    df = df.drop(['last_visit_time','reference_time'],axis = 1)
    return df

In [119]:
def convert_date(df):
    df['joining_date'] = pd.to_datetime(df['joining_date'], format = "%Y-%m-%d")
    const = ['2021-12-31']*no_rows
    df['const'] = const
    df['const'] = pd.to_datetime(df['const'], format = "%Y-%m-%d")
    #df['const']
    df['days_since_joining'] = df['const'] - df['joining_date']    
    df['days_since_joining'] = df['days_since_joining'] / np.timedelta64(1,'D')
    df = df.drop(['joining_date','const'],axis = 1)
    #df
    return df

In [120]:
def Encoder(df):
          columnsToEncode = list(df.select_dtypes(include=['category','object']))
          le = LabelEncoder()
          for feature in columnsToEncode:
              df[feature] = le.fit_transform(df[feature].astype(str))  
              try:
                  df[feature] = le.fit_transform(df[feature])
              except:
                  print('Error encoding '+feature)
          return df

In [121]:
df = convert_time(df)
df = convert_date(df)
df = Encoder(df)
df

,age,gender,region_category,membership_category,joined_through_referral,preferred_offer_types,medium_of_operation,internet_option,days_since_last_login,avg_time_spent,...,avg_frequency_login_days,points_in_wallet,used_special_discount,offer_application_preference,past_complaint,complaint_status,feedback,churn_risk_score,seconds_since_last_visit,days_since_joining
0,18,0,3,3,1,2,0,2,17,300.630000,...,747,781.750000,1,1,0,1,4,2,28317.0,1597.0
1,32,0,1,4,0,2,2,1,16,306.340000,...,733,0.000000,1,0,1,2,5,1,40906.0,1586.0
2,44,0,2,2,2,2,2,2,14,516.160000,...,780,500.690000,0,1,1,3,3,5,3998.0,1876.0
3,37,1,1,2,2,2,2,1,11,53.270000,...,1638,567.660000,0,1,1,4,3,5,28929.0,1889.0
4,31,0,1,2,1,1,3,1,20,113.130000,...,744,663.060000,0,1,1,2,3,5,29595.0,1571.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36987,46,0,0,0,1,1,2,2,2,-650.682759,...,1638,639.510000,0,1,1,0,0,4,71154.0,1562.0
36988,29,0,2,0,1,3,3,2,13,-638.123421,...,810,527.990000,1,0,0,1,1,5,2488.0,2013.0
36989,23,0,0,0,2,2,2,2,12,154.940000,...,683,680.470000,0,1,1,4,3,4,72574.0,1937.0
36990,53,1,3,3,1,2,3,1,15,482.610000,...,775,197.264414,1,1,0,1,0,3,50996.0,1660.0


In [122]:
#Y is the dependent variable or output
Y = df.iloc[:,-3].values
Y
df = df.drop(['churn_risk_score'],axis = 1)
df

,age,gender,region_category,membership_category,joined_through_referral,preferred_offer_types,medium_of_operation,internet_option,days_since_last_login,avg_time_spent,avg_transaction_value,avg_frequency_login_days,points_in_wallet,used_special_discount,offer_application_preference,past_complaint,complaint_status,feedback,seconds_since_last_visit,days_since_joining
0,18,0,3,3,1,2,0,2,17,300.630000,53005.25,747,781.750000,1,1,0,1,4,28317.0,1597.0
1,32,0,1,4,0,2,2,1,16,306.340000,12838.38,733,0.000000,1,0,1,2,5,40906.0,1586.0
2,44,0,2,2,2,2,2,2,14,516.160000,21027.00,780,500.690000,0,1,1,3,3,3998.0,1876.0
3,37,1,1,2,2,2,2,1,11,53.270000,25239.56,1638,567.660000,0,1,1,4,3,28929.0,1889.0
4,31,0,1,2,1,1,3,1,20,113.130000,24483.66,744,663.060000,0,1,1,2,3,29595.0,1571.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36987,46,0,0,0,1,1,2,2,2,-650.682759,27277.68,1638,639.510000,0,1,1,0,0,71154.0,1562.0
36988,29,0,2,0,1,3,3,2,13,-638.123421,11069.71,810,527.990000,1,0,0,1,1,2488.0,2013.0
36989,23,0,0,0,2,2,2,2,12,154.940000,38127.56,683,680.470000,0,1,1,4,3,72574.0,1937.0
36990,53,1,3,3,1,2,3,1,15,482.610000,2378.86,775,197.264414,1,1,0,1,0,50996.0,1660.0


In [123]:
#used here to transform categorical data
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
ct = ColumnTransformer(transformers=[('cat', OneHotEncoder(), ['gender','region_category','membership_category','joined_through_referral','preferred_offer_types','medium_of_operation','internet_option','used_special_discount','offer_application_preference','past_complaint','complaint_status','feedback'])], remainder='passthrough')
X = np.array(ct.fit_transform(df))

In [124]:
X

array([[1.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
        7.81750000e+02, 2.83170000e+04, 1.59700000e+03],
       [1.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
        0.00000000e+00, 4.09060000e+04, 1.58600000e+03],
       [1.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
        5.00690000e+02, 3.99800000e+03, 1.87600000e+03],
       ...,
       [1.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
        6.80470000e+02, 7.25740000e+04, 1.93700000e+03],
       [0.00000000e+00, 1.00000000e+00, 0.00000000e+00, ...,
        1.97264414e+02, 5.09960000e+04, 1.66000000e+03],
       [0.00000000e+00, 1.00000000e+00, 0.00000000e+00, ...,
        7.19970000e+02, 8.04070000e+04, 2.26100000e+03]])